## 🧹 Data Cleaning and Preparation

### Overview
In this step, we clean and prepare the raw datasets (`gdp.csv` and `sp500.csv`) for analysis.  
The goal is to ensure consistency, remove noise, and format both datasets for integration.


### GDP Cleaning

In this section, we load the quarterly GDP data from FRED, clean it, and prepare it for merging:

- Rename columns for consistency (`date`, `gdp`)
- Drop rows with missing or invalid GDP values
- Convert `date` to `datetime` and `gdp` to numeric
- Sort by date
- Resample quarterly GDP to **monthly** frequency and forward-fill missing months  
  (so it aligns with the stock data frequency)


In [16]:
import pandas as pd
import os

raw_dir = "data/raw"
clean_dir = "data/clean"
os.makedirs(clean_dir, exist_ok=True)

# -----------------------------
# 1a. LOAD & CLEAN GDP (quarterly)
# -----------------------------
gdp = pd.read_csv(os.path.join(raw_dir, "gdp.csv"))
gdp.columns = ["date", "gdp"]
gdp = gdp.dropna(subset=["gdp"])
gdp["date"] = pd.to_datetime(gdp["date"], errors="coerce")
gdp["gdp"] = pd.to_numeric(gdp["gdp"], errors="coerce")
gdp = gdp.dropna().sort_values("date")

# Resample GDP to monthly and forward-fill
gdp = gdp.set_index("date").resample("M").ffill().reset_index()


/var/folders/lg/6qn5h0zx1rbbp7z4jx4g54980000gr/T/ipykernel_33055/1082042893.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  gdp = gdp.set_index("date").resample("M").ffill().reset_index()


### Other Macroeconomic Indicators

We clean additional monthly indicators from FRED:

- Consumer Price Index (`CPIAUCSL`)
- Unemployment Rate (`UNRATE`)
- Industrial Production (`INDPRO`)
- Retail Sales (`RSAFS`)

Cleaning steps for each indicator:

1. Rename columns (`date`, indicator name)
2. Convert `date` to datetime and values to numeric
3. Drop rows with missing values
4. Sort by date

These indicators will provide additional economic context for trend analysis.

In [17]:
# -----------------------------
# 1b. LOAD & CLEAN OTHER MACRO INDICATORS (monthly)
# -----------------------------
def clean_macro(filename, col_name):
    df = pd.read_csv(os.path.join(raw_dir, filename))
    df.columns = ["date", col_name]
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df[col_name] = pd.to_numeric(df[col_name], errors="coerce")
    df = df.dropna().sort_values("date")
    return df

cpi = clean_macro("cpi.csv", "cpi")
unemployment = clean_macro("unemployment.csv", "unemployment")
indpro = clean_macro("industrial_production.csv", "indpro")
retail = clean_macro("retail_sales.csv", "retail")

### S&P 500 Cleaning

Here we clean the raw stock data:

- Drop unnecessary metadata rows
- Rename columns (`date`, `open`, `high`, `low`, `close`, `volume`)
- Convert columns to numeric types
- Drop rows with missing date or close price
- Sort by date
- Resample daily stock data to **monthly average** to align with macro indicators


In [18]:
# -----------------------------
# S&P 500 CLEANING
# -----------------------------
sp500_raw = pd.read_csv(os.path.join(raw_dir, "sp500.csv"))
sp500 = sp500_raw.iloc[2:].copy()
sp500.columns = ["date", "close", "high", "low", "open", "volume"]
sp500["date"] = pd.to_datetime(sp500["date"], errors="coerce")
for col in ["close", "high", "low", "open", "volume"]:
    sp500[col] = pd.to_numeric(sp500[col], errors="coerce")
sp500 = sp500.dropna(subset=["date", "close"]).sort_values("date")

# Resample S&P 500 to monthly to align with macro indicators
sp500 = sp500.set_index("date").resample("M").mean().reset_index()


/var/folders/lg/6qn5h0zx1rbbp7z4jx4g54980000gr/T/ipykernel_33055/794922781.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  sp500 = sp500.set_index("date").resample("M").mean().reset_index()


### Merging Stock and Macro Data

We merge the cleaned S&P 500 data with all macroeconomic indicators:

- Use **left joins** to keep all stock data
- Merge GDP, CPI, Unemployment, Industrial Production, and Retail Sales
- Forward/backward fill missing values to maintain continuity
- Resulting dataframe is ready for analysis and modeling


In [19]:
# -----------------------------
# ALIGN DATES + MERGE
# -----------------------------
merged_df = sp500.merge(gdp, on="date", how="left") \
                 .merge(cpi, on="date", how="left") \
                 .merge(unemployment, on="date", how="left") \
                 .merge(indpro, on="date", how="left") \
                 .merge(retail, on="date", how="left")


In [20]:
# -----------------------------
# HANDLE MISSING VALUES
# -----------------------------
merged_df.fillna(method="ffill", inplace=True)
merged_df.fillna(method="bfill", inplace=True)

/var/folders/lg/6qn5h0zx1rbbp7z4jx4g54980000gr/T/ipykernel_33055/2242485129.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method="ffill", inplace=True)
/var/folders/lg/6qn5h0zx1rbbp7z4jx4g54980000gr/T/ipykernel_33055/2242485129.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method="bfill", inplace=True)


### Saving Cleaned Files

All cleaned individual indicators and the merged dataset are saved to `data/clean`:

- `gdp_clean.csv`, `cpi_clean.csv`, `unemployment_clean.csv`, `indpro_clean.csv`, `retail_clean.csv`, `sp500_clean.csv`
- `merged_clean.csv` contains the S&P 500 and all macroeconomic indicators merged and ready for analysis


In [ ]:
# -----------------------------
# SAVE CLEANED FILES
# -----------------------------
gdp.to_csv(os.path.join(clean_dir, "gdp_clean.csv"), index=False)
cpi.to_csv(os.path.join(clean_dir, "cpi_clean.csv"), index=False)
unemployment.to_csv(os.path.join(clean_dir, "unemployment_clean.csv"), index=False)
indpro.to_csv(os.path.join(clean_dir, "indpro_clean.csv"), index=False)
retail.to_csv(os.path.join(clean_dir, "retail_clean.csv"), index=False)
sp500.to_csv(os.path.join(clean_dir, "sp500_clean.csv"), index=False)
merged_df.to_csv(os.path.join(clean_dir, "merged_clean.csv"), index=False)

print("✅ Cleaned files saved to data/clean/")


✅ Cleaned files saved to data/clean/


,date,close,high,low,open,volume,gdp,cpi,unemployment,indpro,retail
0,2015-01-31,2028.178510,2043.618506,2013.377496,2031.392981,3.866502e+09,18063.529,NaN,NaN,NaN,NaN
1,2015-02-28,2082.195794,2086.168952,2069.651046,2076.614727,3.619766e+09,18063.529,NaN,NaN,NaN,NaN
2,2015-03-31,2079.990440,2090.509988,2070.099559,2081.527283,3.485266e+09,18063.529,NaN,NaN,NaN,NaN
3,2015-04-30,2094.862839,2102.731887,2085.031448,2094.028123,3.431473e+09,18279.784,NaN,NaN,NaN,NaN
4,2015-05-31,2111.943518,2119.614966,2103.135986,2111.099500,3.259386e+09,18279.784,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
115,2024-08-31,5478.214577,5512.019576,5440.693182,5473.899969,3.686241e+09,29511.664,NaN,NaN,NaN,NaN
116,2024-09-30,5621.260449,5648.423511,5588.659521,5624.488989,3.978242e+09,29511.664,NaN,NaN,NaN,NaN
117,2024-10-31,5792.319548,5815.695143,5768.833432,5797.090438,3.583149e+09,29825.182,NaN,NaN,NaN,NaN
118,2024-11-30,5929.915991,5948.672974,5899.855493,5920.711963,4.205099e+09,29825.182,NaN,NaN,NaN,NaN
